In [1]:
from numpy.core.numeric import tensordot
import pyrealsense2 as rs
import numpy as np
import cv2
import os
import time
import mediapipe as mp
import math
import csv
import plotly.offline as po
import plotly.graph_objects as go
import pandas as pd
import pyautogui


# 棒人間のライン座標設定用    
def F(i, a, b):
    fig.data[i].x = marker_z[a], marker_z[b]
    fig.data[i].y = marker_x[a], marker_x[b]
    fig.data[i].z = marker_y[a], marker_y[b]
        
po.init_notebook_mode(connected = True)

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# make dir
timestr = time.strftime("%y%m%d-%H%M%S")
dirname = "recordimages/%s" % timestr
dirname2 = "plotlyimages/%s" % timestr
os.makedirs(dirname)
os.makedirs(dirname2)

# ストリーム(IR/Color/Depth)の設定
w = 1280
h = 720
config = rs.config()
config.enable_stream(rs.stream.color, w, h, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, w, h, rs.format.z16, 30)

# ストリーミング開始
pipeline = rs.pipeline()
profile = pipeline.start(config)

# Alignオブジェクト生成
align_to = rs.stream.color
align = rs.align(align_to)

try:
    count = 0

    with open('%s.csv' % timestr, 'w', newline="") as f:
        writer = csv.writer(f)

        # CSVファイルの1行目を書き込み
        csv_header = ["FrameNo.",
                      "0_x", "0_y", "0_z", "1_x", "1_y", "1_z", "2_x", "2_y", "2_z",
                      "3_x", "3_y", "3_z", "4_x", "4_y", "4_z", "5_x", "5_y", "5_z",
                      "6_x", "6_y", "6_z", "7_x", "7_y", "7_z", "8_x", "8_y", "8_z",
                      "9_x", "9_y", "9_z", "10_x", "10_y", "10_z", "11_x", "11_y", "11_z",
                      "12_x", "12_y", "12_z", "13_x", "13_y", "13_z", "14_x", "14_y", "14_z",
                      "15_x", "15_y", "15_z", "16_x", "16_y", "16_z", "17_x", "17_y", "17_z",
                      "18_x", "18_y", "18_z", "19_x", "19_y", "19_z", "20_x", "20_y", "20_z",
                      "21_x", "21_y", "21_z", "22_x", "22_y", "22_z", "23_x", "23_y", "23_z",
                      "24_x", "24_y", "24_z", "25_x", "25_y", "25_z", "26_x", "26_y", "26_z",
                      "27_x", "27_y", "27_z", "28_x", "28_y", "28_z", "29_x", "29_y", "29_z",
                      "30_x", "30_y", "30_z", "31_x", "31_y", "31_z", "32_x", "32_y", "32_z"
                      ]
        writer.writerow(csv_header)
        
        # マーカー座標リスト
        marker_x = [0]
        marker_y = [0]
        marker_z = [0]
        
        # ライン
        data = []
        for i in range(36):
            data.append(go.Scatter3d(
                x =  [0, 0],
                y =  [0, 0],
                z =  [0, 0],
                mode = 'lines+markers',
                marker = dict(
                    color = 'rgb(0, 0, 100)',
                    size = 3,
                    opacity = 0.8
                )))
        fig = go.FigureWidget(data)
                    
        # レイアウト
        fig.update_layout(
            height = 500,
            width = 500,
            margin = dict(
                l = 0,
                r = 0,
                b = 0,
                t = 0
            ),
            #カメラ位置
            scene_camera = dict(
                eye = dict(x = 0, y = 0, z = 1)
            ),
            
            # 軸設定
            scene = dict(
                aspectmode = 'cube',
                xaxis = dict(range = [-10.0, 10.0]),
                yaxis = dict(range = [2.0, -2.0]),
                zaxis = dict(range = [2.0, -2.0])),
                showlegend = False,
            )
        display(fig)
            
        # test.pyより
        with mp_pose.Pose(
                    min_detection_confidence = 0.7,
                    min_tracking_confidence = 0.3) as pose:
            while True:
                flag = True
                XYZlist = []
            
                # フレーム待ち(Color & Depth)
                frames = pipeline.wait_for_frames()
            
                aligned_frames = align.process(frames)
                color_frame = aligned_frames.get_color_frame()
                depth_frame = aligned_frames.get_depth_frame()
                depth_image = np.asanyarray(depth_frame.get_data())
            
                if not depth_frame or not color_frame:
                    continue

                # imageをnumpy arrayに
                color_image = np.asanyarray(color_frame.get_data())
                depth_image = np.asanyarray(depth_frame.get_data())
            
                image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
                
                # To improve performance, optionally mark the image as not writeable to
                # pass by reference.
                image.flags.writeable = False
                results = pose.process(image)

                # Draw the pose annotation on the image.
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                mp_drawing.draw_landmarks(
                    image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

                # pose_landmarksが取得されていなければcontinueで継続
                if not results.pose_landmarks:
                    continue  

                # 取得されていればランドマークの座標を出力
                # 例として鼻（nose）の値を出力、fはフォーマット文字列で、書式を整えて出力できる
                # フォーマット文字列に変数の値を埋め込む場合は、{ }で囲んで埋め込む
                # distは辞書型、x, yはmediapipeから取得、それをもとにZ（深度）はRealSence2から取得
                # dist = {"x": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW].x,
                #      "y": results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW].y}
                # print(dist["x"],dist["y"])

                # FocalLength(X,Y,Z)
                fx = w / (2 * math.tan(math.radians(69.4) / 2))
                fy = h / (2 * math.tan(math.radians(42.5) / 2))

                # CSVファイルの2行目の準備
                XYZlist = [count] 

                # すべてのランドマークについて繰り返し
                for i in range(33):
                    try:
                        lmk = results.pose_landmarks.landmark[i]
                    
                        # ランドマークのxy座標からZ座標を取得する
                        dist = {"x": lmk.x, "y": lmk.y}
                        dist["Z"] = depth_frame.get_distance(
                            int(dist["x"]) * w, int(dist["y"]) * h)

                        X = dist["Z"] / fx * (dist["x"] * w - w / 2)
                        Y = dist["Z"] / fy * (dist["y"] * h - h / 2)
                        Z = dist["Z"]

                    except:
                        # ランドマーク取得不可、または異常値が検出された場合は仮に原点の値を入れておく
                        X = 0
                        Y = 0
                        Z = 0
                        
                    #X->Z Y->X Z->Y
                    XYZlist.extend([Z, X, Y ])
                    
                    # マーカーリストに追加する
                    marker_x.append(X)
                    marker_y.append(Y)
                    marker_z.append(Z)

                # save (X, Y, Z) to CSV ※林君のツール用CSVに合わせる場合は要調整
                writer.writerow(XYZlist)

                # depth imageをカラーマップに変換
                depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(
                    depth_image, alpha = 0.08), cv2.COLORMAP_JET)

                # 画像表示
                color_image_s = cv2.resize(color_image, (640, 360))
                depth_colormap_s = cv2.resize(depth_colormap, (640, 360))
                mp_image_s = cv2.resize(image, (640, 360))

                images_h1 = np.hstack((color_image_s, depth_colormap_s))
                images_h2 = np.hstack((mp_image_s, mp_image_s))
                images_v = np.vstack((images_h1, images_h2))
                cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
                cv2.imshow('RealSense', images_v)

                if cv2.waitKey(1) & 0xff == 27:  # ESCで終了
                    cv2.destroyAllWindows()
                    break    
                #save images
#                 colorfilepath = dirname + "/color%5d.png" % count
#                 depthfilepath = dirname + "/depth%5d.png" % count
#                 mpfilepath = dirname + "/mp%5d.png" % count
#                 cv2.imwrite(colorfilepath, color_image)
#                 cv2.imwrite(depthfilepath, depth_image)
#                 cv2.imwrite(mpfilepath, image)
            
                # グラフを再描画する
                with fig.batch_update():
                    try:
                        # 頭部
                        F(1, 0, 1)
                        F(2, 1, 2)
                        F(3, 2, 3)
                        F(4, 0, 4)
                        F(5, 4, 5)
                        F(6, 5, 6)
                        F(7, 3, 7)
                        F(8, 6, 8)
                        F(10, 9, 10)
                    
                        # 上半身
                        F(12, 11, 12)
                        F(13, 11, 13)
                        F(14, 12, 14)
                        F(15, 13, 15)
                        F(16, 14, 16)
                        F(17, 15, 17)
                        F(18, 16, 18)
                        F(19, 15, 19)
                        F(20, 16, 20)
                        F(21, 15, 21)
                        F(22, 16, 22)
                        F(33, 18, 20)
                        F(34, 17, 19)
                    
                        # 下半身
                        F(23, 11, 23)
                        F(24, 12, 24)
                        F(35, 23, 24)
                        F(25, 23, 25)
                        F(26, 24, 26)
                        F(27, 25, 27)
                        F(28, 26, 28)
                        F(29, 27, 29)
                        F(30, 28, 30)
                        F(31, 27, 31)
                        F(32, 28, 32)
                    
                    except:
                        continue
                # sample ディレクトリ下にtest.pngとして保存する
                plotlyfilepath = dirname2 + "/plotly%5d.png" % count
                #fig.write_image(plotlyfilepath) 
                pyautogui.screenshot(plotlyfilepath)

                
                # マーカーリストを初期化しておく
                marker_x = []
                marker_y = []
                marker_z = []

                count += 1

finally:
    # ストリーミング停止
    pipeline.stop()

FigureWidget({
    'data': [{'marker': {'color': 'rgb(0, 0, 100)', 'opacity': 0.8, 'size': 3},
              '…